In [2]:
import pandas as pd
import csv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
data_iter4 = pd.read_csv('Ava_CleanBW.csv')

In [4]:
data_iter4["Entropy_NonR"] = 0
data_iter4 = data_iter4.astype({"Entropy_NonR": float})
data_iter4

,Unnamed: 0,Unnamed: 0.1,Index,ID,Note1,Note2,Note3,Note4,Note5,Note6,...,Note9,Note10,Tag1,Tag2,ChallengeID,B,Entropy_Color,Entropy_BW,E/E_bw,Entropy_NonR
0,0,10,11,953417,0,0,0,5,32,50,...,3,1,22,0,1396,6.112903,0.0,0.0,0.0,0.0
1,1,11,12,953777,0,3,2,3,13,40,...,8,3,20,53,1396,6.593750,0.0,0.0,0.0,0.0
2,2,12,13,953756,0,2,3,9,35,50,...,2,2,0,0,1396,5.796875,0.0,0.0,0.0,0.0
3,3,13,14,954195,0,1,7,26,56,23,...,0,2,0,0,1396,5.040984,0.0,0.0,0.0,0.0
4,4,14,15,953903,0,1,4,5,33,50,...,3,2,21,28,1396,5.943548,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249752,255503,255525,255526,56938,1,7,17,40,84,56,...,6,1,9,0,181,5.339844,0.0,0.0,0.0,0.0
249753,255504,255526,255527,57304,1,0,8,25,50,65,...,18,7,1,18,181,6.235294,0.0,0.0,0.0,0.0
249754,255505,255527,255528,56711,0,0,8,11,26,47,...,30,22,9,24,181,7.072581,0.0,0.0,0.0,0.0
249755,255506,255528,255529,57303,0,2,19,30,75,60,...,5,2,22,0,181,5.608871,0.0,0.0,0.0,0.0


In [5]:
import timeit

In [6]:
def entropy_bw(I):
    #p = np.array([(I == x).sum() for x in range(256)]) #calcul d'histogramme
    p, _ = np.histogram(I, bins = 256)
    p = p/p.sum() #normalisation de l'histogramme
    #ent_I = -(np.sum((p+eps)*np.log2(p+eps))) #calcul de l'entropie
    ent_I = -(np.sum(p[p>0]*np.log2(p[p>0]))) #calcul de l'entropie
    return ent_I

def entropy_color(I, k):
    I = I/k #Réduire la dynamique de l'image en la divisant par k
    I = I.astype(int) # Ne garder que la partie entière des valeurs
    p_values = int(256/k)
    H_3d = np.zeros((p_values,p_values,p_values)) # Initialiser l'histogramme 3D par la nouvelle dynamique
    
    for i in range(I.shape[0]):
        for j in range(I.shape[1]):
            x = I[i,j] # x est un pixel qui a trois valeurs R G B
            H_3d[x[0],x[1],x[2]] += 1 #on va incrémenter le nombre des valeurs R G B sur l'histogramme3D
            
    p = H_3d/H_3d.sum() # On normalise l'histogramme
    
    #entropy = -(np.sum((p+eps)*np.log2(p+eps))) #calcul de l'entropie
    entropy = -(np.sum(p[p>0]*np.log2(p[p>0]))) #calcul de l'entropie

    return entropy

In [7]:
start = timeit.default_timer()


for i in range(20):
    I = np.array(Image.open('images/' + str(data_iter4["ID"][i]) + '.jpg'))
    Ibw = I.mean(axis = 2)
    e1 = entropy_bw(Ibw)
    e2 = entropy_color(I,4)
    data_iter4.at[i, 'Entropy_BW'] = e1
    data_iter4.at[i, 'Entropy_Color'] = e2
    print(i)
    

stop = timeit.default_timer()
execution_time = stop - start
print("--- %s seconds ---" % (execution_time))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
--- 8.756192700000001 seconds ---


In [ ]:
n = data_iter4["ID"].size
for i in range(51000):
    I = np.array(Image.open('images/' + str(data_iter4["ID"][i]) + '.jpg'))
    Ibw = bw_Image(I)
    e1 = entropy_bw(Ibw)
    e2 = entropy_color(I,4)
    data_iter4.at[i, 'Entropy_BW'] = e1
    data_iter4.at[i, 'Entropy_Color'] = e2
    print(i)

data_iter4.to_csv("AVA_Entropies_Calculated.csv")